In [1]:
from abc import ABC, abstractmethod
import copy


class val:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

    def __eq__(self, other):
        try:
            if self.name == other.name:
                return True
        except:
            return False
        return False

    def isval(self):
        return True

    def istuyen(self):
        return False

    def ishoi(self):
        return False

    def isphu(self):
        return False


class menhde:
    def __init__(self, val1, val2):
        pass

    def __str__(self):
        pass

    def istuyen(self):
        return False

    def ishoi(self):
        return False

    def isphu(self):
        return False

    @abstractmethod
    def __eq__(self, other):
        # phải cùng kiểu hội hoặc tuyển
        condition1 = (self.istuyen() and other.istuyen()) or (self.ishoi() and other.ishoi())
        if not condition1:
            return False

        # 2 vế bằng nhau
        condition2 = self.val1 == other.val1 and self.val2 == other.val2
        condition3 = self.val1 == other.val2 and self.val2 == other.val1
        if (condition2 or condition3):
            return True

        return False


class hoi(menhde):
    def __init__(self, val1, val2):
        self.val1 = val1
        self.val2 = val2

    def __str__(self):
        return "(" + str(self.val1) + "^" + str(self.val2) + ")"

    def ishoi(self):
        return True

    def phanra(self):  # a^b chuyển thành a,b
        return [self.val1, self.val2]  # trả về 2 giá trị


class tuyen(menhde):
    def __init__(self, val1, val2):
        self.val1 = val1
        self.val2 = val2

    def __str__(self):
        return "(" + str(self.val1) + "v" + str(self.val2) + ")"

    def istuyen(self):
        return True

    def phanra(self):
        return [self.val1, self.val2]


class phu(menhde):
    def __init__(self, val):
        self.val = val

    def __str__(self):
        return "!" + str(self.val)

    def isphu(self):
        return True

    def __eq__(self, other):
        try:
            if self.val == other.val:
                return True
        except:
            return False
        return False

    def isval(self):
        return True


class bieuthuc:
    # Bước 1 : khởi tạo dưới dạng chuẩn
    def __init__(self, vetrai, vephai, prev=None):
        self.prev = prev  # Node biểu thức trước đó
        self.vetrai = vetrai  # list các mệnh đề vế trái
        self.vephai = vephai  # list các mệnh đề vế phải

    # Bước 2 : chuyển vế đổi dấu
    def chuyenve(self):
        # nếu vế trái có mệnh đề phủ định thì chuyển sang vế phải và đổi dấu
        for menhde in self.vetrai:
            if menhde.isphu():
                self.vephai.append(menhde.val)
                self.vetrai.remove(menhde)
        # nếu vế phải có mệnh đề phủ định thì chuyển sang vế trái và đổi dấu
        for menhde in self.vephai:
            if menhde.isphu():
                self.vetrai.append(menhde.val)
                self.vephai.remove(menhde)


    #Bước 3 : thay các dấu hội ở giả thiết và dấu tuyển ở kết luận bằng dấu phẩy
    def phanra(self):
        flag = 0
        # các biểu thức ở vế trái nếu là phép hội thì phân ra thành 2 giá trị
        for menhde in self.vetrai:
            if menhde.ishoi():
                self.vetrai += menhde.phanra()
                self.vetrai.remove(menhde)
                flag = 1
        for menhde in self.vephai:
            if menhde.istuyen():
                self.vephai += menhde.phanra()
                self.vephai.remove(menhde)
                flag = 1
        if flag == 1:
            print('phân rã: ', self)
        else:
            print("không phân rã được ", self)
        if (self.checkcm() == 1):
            print('(ĐCM)')
        elif (self.checkcm() == -1):
            print('(Không ĐCM)')
        else:
            print('\n')

    def checkcm(self):
        self.chuyenve()
        #1 là được chứng minh, -1 là không được chứng minh, 0 là còn chứng minh
        for menhdevt in self.vetrai:
            for menhdevp in self.vephai:
                if menhdevt.__eq__(menhdevp) :
                    return 1
        donvi = True #check xem tất cả mệnh đề đã ở dạng đơn vị
        try:
            for menhdevt in self.vetrai:
                for menhdevp in self.vephai:
                    if not menhdevt.isval() or not menhdevp.isval(): #nếu không phải dạng đơn vị thì chuyển sang except
                        pass
        except:
            donvi = False
        if donvi:
            return -1
        return 0

    #Hàm tách biểu thức thành 2 trạng thái để tiếp tục chứng minh
    def chuyentrangthai(self):
        tt1 = None
        tt2 = None
        temp = copy.deepcopy(self)
        temp.prev = temp
        #Bước 1: phân rã
        temp.phanra()
        #Bước 2: chuyển vế
        temp.chuyenve()
        #Bước 3: tách thành 2 trạng thái ở vế trái
        for menhdevt in temp.vetrai:
            if menhdevt.istuyen():
                temp.vetrai.remove(menhdevt)
                tt1 = copy.deepcopy(temp)
                tt1.vetrai.append(menhdevt.phanra()[0])
                tt2 = copy.deepcopy(temp)
                tt2.vetrai.append(menhdevt.phanra()[1])
                return [tt1,tt2]
        # Bước 3: tách thành 2 trạng thái ở vế phải
        for menhdevp in temp.vephai:
            if menhdevp.ishoi():
                temp.remove(menhdevp)
                tt1.copy.deepcopy(self)
                tt1.vephai.append(menhdevp.phanra()[0])
                tt2.copy.deepcopy(self)
                tt2.vephai.append(menhdevp.phanra()[1])
                return [tt1,tt2]
        return [tt1,tt2]


    def __str__(self):
        vetraistr = ",".join([str(menhde) for menhde in self.vetrai])
        vephaistr = ",".join([str(menhde) for menhde in self.vephai])
        return vetraistr + " -> " + vephaistr

class VuongHao:
    def __init__(self, bieuthuc):
        self.Nodes = [bieuthuc]

    def checktt(self):
        if -1 in (bieuthuc.checkcm() for bieuthuc in self.Nodes):
            return -1
        if 0 in (bieuthuc.checkcm() for bieuthuc in self.Nodes):
            return 0
        return 1

    def run(self):
        print('Biểu thức của chúng ta là: ', self.Nodes[0])
        while(self.checktt() == 0):
            for bieuthuc in self.Nodes:
                self.Nodes.remove(bieuthuc)
                tt1,tt2 = bieuthuc.chuyentrangthai()
                if tt1 == None:
                    continue
                else:
                    self.Nodes.append(tt1)
                    self.Nodes.append(tt2)
                    print('Tách ' , str(bieuthuc) , " thành " , tt1 , " và " ,tt2)
            if self.checktt() == -1:
                break
        print('Kết quả:', end = " ")
        if self.checktt() == 1 : print('Được chứng minh')
        if self.checktt() == -1: print('Không được chứng minh')
        for bieuthuc in self.Nodes:
            print(str(bieuthuc))




menhde1 = tuyen(phu(val('a')),val('b'))
menhde2 = tuyen(phu(val('b')),val('c'))
menhde3 = tuyen(phu(val('a')),val('c'))
mybieuthuc = bieuthuc([menhde1,menhde2],[menhde3])
VuongHao(mybieuthuc).run()


Biểu thức của chúng ta là:  (!avb),(!bvc) -> (!avc)
phân rã:  (!avb),(!bvc) -> !a,c


Tách  (!avb),(!bvc) -> (!avc)  thành  (!bvc),a,!a -> c  và  (!bvc),a,b -> c
không phân rã được  (!bvc),a,b -> c


Tách  (!bvc),a,b -> c  thành  a,b,!b -> c  và  a,b,c -> c
không phân rã được  a,b,c -> c
(ĐCM)
Kết quả: Được chứng minh
(!bvc),a -> c,a
a,b -> c,b
